## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.650,115.3333,62.01,65,97,13.02,overcast clouds
1,1,Hami,CN,42.800,93.4500,79.25,11,0,2.82,clear sky
2,2,Ushuaia,AR,-54.800,-68.3000,37.06,87,100,26.46,light intensity drizzle
3,3,Qaanaaq,GL,77.484,-69.3632,24.89,83,41,8.66,scattered clouds
4,4,Sainte-Maxime,FR,43.300,6.6333,69.82,35,24,30.11,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hami,CN,42.8000,93.4500,79.25,11,0,2.82,clear sky
7,7,Bathsheba,BB,13.2167,-59.5167,83.01,83,40,17.27,scattered clouds
11,11,Butaritari,KI,3.0707,172.7902,81.86,70,18,13.06,few clouds
12,12,Kapaa,US,22.0752,-159.3190,80.58,84,0,8.05,clear sky
13,13,Tabuk,PH,17.4189,121.4443,87.06,65,74,1.86,broken clouds
16,16,Cayenne,GF,4.9333,-52.3333,75.24,94,0,0.00,clear sky
22,22,Meulaboh,ID,4.1363,96.1285,84.38,65,86,5.70,overcast clouds
23,23,Oranjestad,AW,12.5240,-70.0270,83.62,77,97,16.91,overcast clouds
24,24,Vaini,TO,-21.2000,-175.2000,75.36,88,100,10.36,overcast clouds
28,28,Hilo,US,19.7297,-155.0900,78.28,88,20,3.44,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows.

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hami,CN,79.25,clear sky,42.8000,93.4500,
7,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,
11,Butaritari,KI,81.86,few clouds,3.0707,172.7902,
12,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
13,Tabuk,PH,87.06,broken clouds,17.4189,121.4443,
16,Cayenne,GF,75.24,clear sky,4.9333,-52.3333,
22,Meulaboh,ID,84.38,overcast clouds,4.1363,96.1285,
23,Oranjestad,AW,83.62,overcast clouds,12.5240,-70.0270,
24,Vaini,TO,75.36,overcast clouds,-21.2000,-175.2000,
28,Hilo,US,78.28,few clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
filter = hotel_df["Hotel Name"] != ""
clean_hotel_df = hotel_df[filter]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hami,CN,79.25,clear sky,42.8000,93.4500,Laoyutang Resort
7,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,Atlantis Hotel
11,Butaritari,KI,81.86,few clouds,3.0707,172.7902,Isles Sunset Lodge
12,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Tabuk,PH,87.06,broken clouds,17.4189,121.4443,Golden Berries Hotel and Convention
...,...,...,...,...,...,...,...
684,Kamina,CD,83.46,few clouds,-8.7386,24.9906,LEANIF HOTEL RESTAURANT
686,Zhangye,CN,79.95,few clouds,38.9342,100.4517,Zhangye Lvzhou Holiday Inn
687,Dharchula,IN,80.26,overcast clouds,29.8500,80.5333,Mahara Hotel
688,Pedasi,PA,78.06,overcast clouds,7.5333,-80.0333,Hotel Casa de Campo Pedasí


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))